**3. Best Practice ในการเปรียบเทียบด้วย Hashing**

วิธีการ Hash เป็นวิธีที่ มีประสิทธิภาพสูงมาก ในการตรวจสอบการเปลี่ยนแปลงของหลายๆ คอลัมน์พร้อมกัน เพราะคุณลดการเปรียบเทียบจาก N คอลัมน์ เหลือเพียง 1 คอลัมน์ (ค่า Hash)

ขั้นตอน:
1. ระบุ SCD Type 2 Attributes: กำหนดให้ชัดเจนว่าคอลัมน์ใดบ้างที่จะใช้ในการตรวจสอบการเปลี่ยนแปลง
2. สร้าง Hash Column ใน Source Data:
    - รวมค่าของ SCD Type 2 Attributes ทั้งหมด ใน Source Data เข้าด้วยกันเป็น String เดียว

    - ใช้ฟังก์ชัน Hash (เช่น md5, sha2) เพื่อสร้าง Hash Value ของ String นั้น

    - สำคัญ: ตรวจสอบให้แน่ใจว่าการรวม String จัดการ NULL values ได้อย่างสอดคล้องกัน (เช่น coalesce(column, 'NULL_VALUE_PLACEHOLDER')) และเรียงลำดับคอลัมน์ที่จะรวมเหมือนกันทุกครั้ง

In [0]:
from pyspark.sql.functions import concat_ws, md5, col, lit, current_date

# สมมติ source_df มีคอลัมน์ CustomerID, CustomerName, CustomerAddress, CustomerPhone
# และเราจะติดตาม CustomerName, CustomerAddress, CustomerPhone

# คอลัมน์ที่จะใช้สร้าง Hash (SCD Type 2 Attributes)
scd_cols = ["CustomerName", "CustomerAddress", "CustomerPhone"]

# สร้าง Hash Column ใน Source
source_df_hashed = source_df.withColumn(
    "scd_hash",
    md5(concat_ws("||", *[col(c) for c in scd_cols]))
)

**คำแนะนำ: ใช้ concat_ws เพื่อรวม String โดยใช้ตัวคั่นที่ไม่น่าจะปรากฏในข้อมูลจริง (เช่น ||) และถ้ามี NULL values ควรใช้ coalesce หรือ nvl เพื่อแทนที่ NULL ด้วยค่า placeholder ที่สอดคล้องกัน เช่น coalesce(col("CustomerAddress"), lit("NULL_ADDRESS")) เพื่อให้ Hash ไม่เปลี่ยนเมื่อค่า Null เปลี่ยน**

**3. เตรียม Destination Data:**

- โหลด Destination Dimension Table (เช่น dim_customer)
- กรองเฉพาะ Record ปัจจุบัน (IsCurrent = True)
- สร้าง Hash Column สำหรับ Record ปัจจุบันเหล่านี้ด้วย Logic เดียวกันกับ Source Data (ใช้ SCD Type 2 Attributes เดียวกันและวิธี Hash เดียวกัน)

In [0]:
from delta.tables import DeltaTable

# โหลด Dimension Table
dim_customer_table = DeltaTable.forName(spark, "dim_customer")
dim_customer_df = dim_customer_table.toDF().filter(col("IsCurrent") == True)

# สร้าง Hash Column ใน Destination (Current Records)
dim_customer_current_hashed = dim_customer_df.withColumn(
    "scd_hash",
    md5(concat_ws("||", *[col(c) for c in scd_cols])) # ใช้ scd_cols เดียวกัน
)

**4. หาประเภทของ Record (New, Changed, Unchanged):**

- New Records: LEFT ANTI JOIN Source Hashed กับ Destination Hashed (โดยใช้ Business Key)

- Changed Records: INNER JOIN Source Hashed กับ Destination Hashed (โดยใช้ Business Key) และ Source.scd_hash != Destination.scd_hash

- Unchanged Records: INNER JOIN Source Hashed กับ Destination Hashed (โดยใช้ Business Key) และ Source.scd_hash == Destination.scd_hash (คุณสามารถ DO NOTHING กับกลุ่มนี้ หรือ UPDATE LastUpdatedTimestamp ถ้าจำเป็น)

In [0]:
# 1. New Records (Records ใน Source ที่ไม่มีใน Destination Current)
new_records = source_df_hashed.join(
    dim_customer_current_hashed,
    source_df_hashed.CustomerID == dim_customer_current_hashed.CustomerID,
    "left_anti"
)

# 2. Changed Records (Records ที่ Business Key ตรงกัน แต่ Hash ไม่ตรง)
changed_records = source_df_hashed.join(
    dim_customer_current_hashed,
    (source_df_hashed.CustomerID == dim_customer_current_hashed.CustomerID) & \
    (source_df_hashed.scd_hash != dim_customer_current_hashed.scd_hash),
    "inner"
).select(source_df_hashed["*"], dim_customer_current_hashed.SK.alias("old_sk")) # เอา SK เก่ามาด้วยเพื่ออัปเดต

**ดำเนินการ MERGE INTO (ใช้ Delta Lake):**

นี่คือส่วนที่มีประสิทธิภาพที่สุดในการจัดการ SCD Type 2 ด้วย PySpark บน Databricks เพราะ Delta Lake MERGE รองรับทั้ง UPDATE และ INSERT ในคำสั่งเดียว

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date, date_sub

# โหลด Dimension Table เป็น DeltaTable object
dim_customer_delta_table = DeltaTable.forName(spark, "dim_customer")

# กำหนดวันที่ปัจจุบันสำหรับการทำ SCD Type 2
current_date_val = current_date()
yesterday_date_val = date_sub(current_date(), 1)

# 1. อัปเดต Record เก่า (สำหรับ Changed Records)
# เราจะใช้ changed_records_to_update เป็น DataFrame ที่มีคอลัมน์ SK ของ Record เก่าที่ต้องเปลี่ยน EndDate
# ในตัวอย่างนี้ เราจะ MERGE โดยใช้ Business Key จาก changed_records
# และใน WHEN MATCHED เราจะอัปเดต Record เก่าให้ EndDate = current_date - 1 และ IsCurrent = False
# และ INSERT Record ใหม่
# นี่คือขั้นตอนที่ซับซ้อนที่สุดใน MERGE สำหรับ SCD Type 2

# เตรียม Source DataFrame สำหรับ MERGE
# Source สำหรับ MERGE จะรวม New Records และ Changed Records ใหม่
# โดยเพิ่มคอลัมน์ metadata ที่จำเป็นสำหรับ SCD Type 2
# และคอลัมน์สำหรับ Join กับ dim_customer เพื่อหา old_sk สำหรับการอัปเดต

# สำหรับ New Records: เพิ่มคอลัมน์ SCD metadata
new_records_for_merge = new_records.withColumn("StartDate", current_date_val) \
                                   .withColumn("EndDate", lit("9999-12-31").cast("date")) \
                                   .withColumn("IsCurrent", lit(True)) \
                                   .withColumn("scd_action", lit("NEW")) # เพิ่มคอลัมน์ระบุ Action

# สำหรับ Changed Records: เพิ่มคอลัมน์ SCD metadata สำหรับ Record ใหม่
# และระบุ Old SK เพื่อใช้ในการอัปเดต Record เก่า
changed_records_for_merge = changed_records.withColumn("StartDate", current_date_val) \
                                           .withColumn("EndDate", lit("9999-12-31").cast("date")) \
                                           .withColumn("IsCurrent", lit(True)) \
                                           .withColumn("scd_action", lit("CHANGED")) # เพิ่มคอลัมน์ระบุ Action

# รวม New และ Changed records เพื่อเป็น Source เดียวสำหรับ MERGE
# ต้องแน่ใจว่ามีคอลัมน์ครบถ้วนตามที่ Dimension Table คาดหวัง
# ในทางปฏิบัติ เราจะสร้าง DataFrame ที่รวมข้อมูลที่จะถูก INSERT (New records และ New version of changed records)
# และอีก DataFrame ที่ระบุข้อมูลที่จะถูก UPDATE (Old version of changed records)

# วิธีการทั่วไปในการใช้ MERGE สำหรับ SCD Type 2:
# 1. สร้าง Temp View ของ source_df_hashed
# 2. ทำ MERGE INTO destination_table AS target USING source_df_hashed AS source
#    ON target.BusinessKey = source.BusinessKey AND target.IsCurrent = TRUE
#    WHEN MATCHED AND target.scd_hash != source.scd_hash THEN  -- กรณีเปลี่ยนแปลง
#        UPDATE SET EndDate = current_date - 1, IsCurrent = FALSE -- อัปเดต Record เดิม
#    WHEN NOT MATCHED THEN -- กรณี Record ใหม่
#        INSERT * -- ใส่ Record ใหม่
# 3. ต้องรัน MERGE อีกครั้งเพื่อ INSERT Record ใหม่สำหรับ Changed Records

# ด้วย Databricks Delta Lake และ PySpark, Best Practice คือใช้ Python API ของ DeltaTable
# ซึ่งช่วยให้จัดการ Logic ได้ง่ายขึ้น

# Step 1: สร้าง DataFrame ที่รวมข้อมูลที่จะถูก 'MERGE' เข้าไปใน Dimension Table
# นี่คือ Source DataFrame ที่จะใช้ในคำสั่ง MERGE
# มันควรจะมีข้อมูลสำหรับทั้ง NEW RECORDS และ NEW VERSION ของ CHANGED RECORDS
# และคอลัมน์ที่จำเป็นสำหรับการระบุ OLD RECORDS เพื่อ UPDATE (เช่น Business Key)
# และคอลัมน์ metadata สำหรับ SCD Type 2 (StartDate, EndDate, IsCurrent)
source_df_for_merge = source_df_hashed.select(
    col("CustomerID"), # Business Key
    col("CustomerName"), col("CustomerAddress"), col("CustomerPhone"), # SCD Type 2 Attributes
    col("scd_hash") # Hash
    # ... เพิ่มคอลัมน์อื่นๆ ที่เป็น Non-SCD attributes ด้วย
)

# ทำ MERGE
dim_customer_delta_table.alias("target").merge(
    source_df_for_merge.alias("source"),
    """target.CustomerID = source.CustomerID AND target.IsCurrent = TRUE""" # เงื่อนไข Join
) \
.whenMatchedUpdate(
    condition = "target.scd_hash != source.scd_hash", # ถ้า Hash ไม่ตรง (มีการเปลี่ยนแปลง)
    set = { # อัปเดต Record เก่า
        "EndDate": f"CAST('{current_date_val}' AS DATE) - INTERVAL 1 DAY", # ตั้ง EndDate เป็นวันก่อนหน้า
        "IsCurrent": "FALSE"
    }
) \
.whenNotMatchedInsert(
    values = { # Insert Record ใหม่ (สำหรับ New Records)
        "CustomerID": "source.CustomerID",
        "CustomerName": "source.CustomerName",
        "CustomerAddress": "source.CustomerAddress",
        "CustomerPhone": "source.CustomerPhone",
        # ... เพิ่มคอลัมน์อื่นๆ
        "scd_hash": "source.scd_hash",
        "StartDate": f"CAST('{current_date_val}' AS DATE)",
        "EndDate": "CAST('9999-12-31' AS DATE)",
        "IsCurrent": "TRUE"
    }
) \
.execute()

# **สำคัญ:** MERGE API ของ DeltaTable มีข้อจำกัดเล็กน้อยสำหรับ SCD Type 2 แบบมาตรฐาน
# ที่ต้อง UPDATE แถวเดิม และ INSERT แถวใหม่ในรอบเดียวกัน
# คุณอาจต้องรัน MERGE สองครั้ง หรือใช้วิธีที่ซับซ้อนขึ้น
# หรือใช้วิธีที่นิยมคือ:
# Step 1: ระบุ Records ที่ต้อง 'Expire' (Current Records ที่มีการเปลี่ยนแปลง)
# Step 2: ระบุ Records ที่ต้อง 'Insert' (New Records + New Version ของ Changed Records)
# Step 3: รัน DeltaTable.update() สำหรับ Step 1
# Step 4: รัน DeltaTable.insert() สำหรับ Step 2
# หรือใช้ SQL MERGE INTO หลายเงื่อนไข (ซับซ้อนกว่า PySpark API)

# **วิธีการที่แนะนำและชัดเจนกว่าสำหรับการทำ SCD Type 2 โดยเฉพาะ**
# คือการใช้การ Join และการเขียนผลลัพธ์ลง Delta Table แบบ Two-step MERGE/UPDATE-INSERT
# หรือใช้ Logic ที่ซับซ้อนขึ้นใน Single MERGE

# Logic ที่นิยมทำสำหรับการ MERGE SCD Type 2 ใน Databricks:
# 1. ระบุ Record ที่มีอยู่ใน Source แต่ไม่มีใน Target (New)
# 2. ระบุ Record ที่มีอยู่ใน Source และ Target และ Hash ไม่ตรงกัน (Changed)
# 3. ระบุ Record ที่มีอยู่ใน Source และ Target และ Hash ตรงกัน (Unchanged)
# 4. สร้าง DataFrame ที่ประกอบด้วย:
#    a. New Records (จาก 1.)
#    b. New Version of Changed Records (จาก 2.) - มี StartDate = current_date, EndDate = Sentinel, IsCurrent = True
# 5. ใช้ SQL MERGE หรือ DeltaTable.merge() เพื่อ:
#    a. เมื่อ Business Key ตรงกันและ IsCurrent = TRUE:
#       i. ถ้า Hash ไม่ตรง: UPDATE Record ใน Dimension Table นั้นให้ IsCurrent = FALSE และ EndDate = current_date - 1
#       ii. ถ้า Hash ตรง: DO NOTHING หรือ UPDATE LastUpdatedTimestamp
#    b. เมื่อ Business Key ไม่ตรง: INSERT Record (จาก 4.a และ 4.b)

# เนื่องจาก PySpark DeltaTable.merge() มี .whenMatchedUpdate() และ .whenNotMatchedInsert()
# มันไม่สามารถทำ 'Update' และ 'Insert' Record ใหม่ของสิ่งที่ 'Matched' และ 'Changed' ได้ในคำสั่งเดียว
# ดังนั้น สิ่งที่นิยมทำคือ:

# 1. หา Records ที่มีการเปลี่ยนแปลงและต้อง 'Expired'
records_to_expire = dim_customer_current_hashed.join(
    source_df_hashed,
    (dim_customer_current_hashed.CustomerID == source_df_hashed.CustomerID) & \
    (dim_customer_current_hashed.scd_hash != source_df_hashed.scd_hash),
    "inner"
).select(dim_customer_current_hashed.SK) # เอา SK ของ Record เก่าที่ต้องเปลี่ยน EndDate

# 2. อัปเดต Record เก่าให้ 'Expired'
if records_to_expire.count() > 0:
    dim_customer_delta_table.alias("target").merge(
        records_to_expire.alias("source"),
        "target.SK = source.SK"
    ) \
    .whenMatchedUpdate(
        set = {
            "EndDate": f"CAST('{current_date_val}' AS DATE) - INTERVAL 1 DAY",
            "IsCurrent": "FALSE"
        }
    ) \
    .execute()

# 3. หา Records ที่เป็น 'New' หรือ 'Changed' (เวอร์ชันใหม่) ที่ต้อง 'Insert'
#    New: Business Key ไม่มีใน dim_customer_current_hashed
#    Changed: Business Key มีใน dim_customer_current_hashed แต่ hash ไม่ตรง
records_to_insert = source_df_hashed.join(
    dim_customer_current_hashed,
    (source_df_hashed.CustomerID == dim_customer_current_hashed.CustomerID) & \
    (source_df_hashed.scd_hash == dim_customer_current_hashed.scd_hash), # ใช้ Hash ตรงกัน
    "left_anti" # เอาเฉพาะ records ที่ไม่เจอ (new) หรือเจอแต่ Hash ไม่ตรง (changed)
).select(
    col("CustomerID"),
    col("CustomerName"),
    col("CustomerAddress"),
    col("CustomerPhone"),
    # ... คอลัมน์อื่นๆ
    col("scd_hash")
) \
.withColumn("StartDate", current_date_val) \
.withColumn("EndDate", lit("9999-12-31").cast("date")) \
.withColumn("IsCurrent", lit(True))

# 4. Insert Records ใหม่ (ทั้ง New และ New Version of Changed)
if records_to_insert.count() > 0:
    records_to_insert.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .saveAsTable("dim_customer")

**4. ข้อควรพิจารณาเพิ่มเติม**

- Performance ของ Hash: การ Hash เป็นวิธีที่ดี แต่ต้องแน่ใจว่าการรวม String และการคำนวณ Hash ไม่เป็น Bottleneck หากมีหลายร้อยคอลัมน์ที่ต้อง Hash อาจจะต้องพิจารณา Subset ของคอลัมน์ที่สำคัญจริงๆ

- Hash Collision: โอกาสเกิด Hash Collision (สอง String ที่ต่างกันได้ Hash Value เดียวกัน) มีน้อยมากสำหรับ MD5/SHA256 แต่ก็เป็นไปได้ในทางทฤษฎี สำหรับข้อมูลที่มีความสำคัญสูงมากๆ อาจจะต้องพิจารณาความเสี่ยง

- การจัดการ NULL Values: ย้ำอีกครั้งว่าการจัดการ NULL ในคอลัมน์ที่จะ Hash ต้องสอดคล้องกัน มิฉะนั้นการเปลี่ยนแปลงจาก NULL เป็นค่า หรือจากค่าเป็น NULL จะไม่ถูกตรวจจับ หรือ Hash จะไม่ตรงกันเมื่อไม่ควรจะเป็น
 
- เช่น concat_ws("||", coalesce(col("col1"), lit("NULL")), coalesce(col("col2"), lit("NULL")))

- Column Order: ตรวจสอบให้แน่ใจว่าลำดับของคอลัมน์ที่ใช้ในการ concat_ws (หรือการสร้าง Hash) นั้น เหมือนกันทั้งใน Source และ Destination มิฉะนั้น Hash จะไม่ตรงกันแม้ข้อมูลจะเหมือนกัน

- Datatype Consistency: ตรวจสอบว่า Datatype ของคอลัมน์ที่จะ Hash เหมือนกันทั้ง Source และ Destination

- Non-SCD Attributes: คอลัมน์ที่ไม่ติดตามการเปลี่ยนแปลง (เช่น LastUpdatedTimestamp ของ Source) ควรถูก UPDATE ลงใน Record ปัจจุบันของ Destination (ที่ IsCurrent = True) โดยไม่สร้าง Record ใหม่

- ใช้ Delta Lake: SCD Type 2 มีประสิทธิภาพสูงสุดเมื่อใช้ Delta Lake เป็น Destination Table เพราะรองรับ MERGE INTO และ ACID transactions

การใช้ Hashing เป็น Best Practice ที่แข็งแกร่งสำหรับการทำ SCD Type 2 ครับ มันช่วยลดความซับซ้อนของการเปรียบเทียบหลายคอลัมน์ และทำให้โค้ดมีความยืดหยุ่นมากขึ้นเมื่อมีการเพิ่ม/ลบคอลัมน์ SCD ในอนาคตครับ
